<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Purpose-Statement:" data-toc-modified-id="Purpose-Statement:-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Purpose Statement:</a></span></li></ul></li><li><span><a href="#Plot-selectively" data-toc-modified-id="Plot-selectively-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Plot selectively</a></span><ul class="toc-item"><li><span><a href="#Choos-what-you-want" data-toc-modified-id="Choos-what-you-want-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Choos what you want</a></span></li><li><span><a href="#Plot-a-single-Sectors-with-a-specific-ticker" data-toc-modified-id="Plot-a-single-Sectors-with-a-specific-ticker-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Plot a single Sectors with a specific ticker</a></span></li><li><span><a href="#Plot-a-single-TICKER-AND-ITS-INDUSTRY-or-A-INDUSTRY-WITHOUT-SPECIFIC-TICKER" data-toc-modified-id="Plot-a-single-TICKER-AND-ITS-INDUSTRY-or-A-INDUSTRY-WITHOUT-SPECIFIC-TICKER-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Plot a single TICKER AND ITS INDUSTRY or A INDUSTRY WITHOUT SPECIFIC TICKER</a></span></li></ul></li><li><span><a href="#Plot-the-generla-Overview" data-toc-modified-id="Plot-the-generla-Overview-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Plot the generla Overview</a></span><ul class="toc-item"><li><span><a href="#Choose-Sector-and-plot-ALL-Industries-within-sector" data-toc-modified-id="Choose-Sector-and-plot-ALL-Industries-within-sector-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Choose Sector and plot ALL Industries within sector</a></span></li></ul></li></ul></div>

# Purpose Statement:

Ausgehend von der Zeitreihe der Fintess Rankings der 3600 Snp Stocks wirde ermittelt, welches Ranking  
Percentil sich wie in Future Returns manifestiert. Sind also etwa die Top 100 Fitness Ranking Stocks auch die  
Leader (oder LAggards) der naechsten N-day returns ?  
Es wird eine Statistik gebaut, die zeigen soll, ob ein Zusammenhang zw. Ranking und Return besteht.[Update 08 Dez. 2024]



In [5]:
# Run me !
import pandas as pd
import math

import pickle

import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
from IPython.display import display, HTML
from datetime import datetime
# from datetime import date

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

#import dataframe_image as dfi

from matplotlib.backends.backend_pdf import PdfPages


import os
import sys
currentdir = os.path.dirname("")
parentdir = os.path.realpath(os.path.join(currentdir, '..'))
sys.path.insert(0, parentdir) 
sys.path.append(os.path.abspath(os.path.join(currentdir, '../TC')))

import signalgenerator as sg
import underly  as ul

#############################################################
#print(parentdir)
from lib import FileStuff
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

plt.rcParams['axes.grid'] = True

css = """
.output {
    flex-direction: column;
}
"""
HTML('<style>{}</style>'.format(css))

# Stock in major Sector ETF


def color_columns(column):
    
    abs_max=abs(column.max())
    abs_min=abs(column.min())
    maxdist = abs(abs_max-abs_min)
    
    
    if abs_max> abs_min:
       min_adj=column.min()-maxdist
       max_adj=column.max()
    else:
       max_adj=column.max()+maxdist
       min_adj=column.min()
    ## Normiere so, dass Wert Null gneua in Mitte liegt "
    norm = plt.Normalize(min_adj,max_adj)
    colors = plt.cm.RdYlGn(norm(column))
    return ['background-color: {}'.format(plt.cm.colors.rgb2hex(color)) for color in colors]

#energy_US=
#communicationservices_US=
#industrials_US=
#realsestate_US=
#materials_US=
#financials_US=
#consumercyclical_US=

# utilities_EU=
# healthcare_EU =
# consumerdefensive_EU=
# technology_EU=
# #energy_EU=
# #communicationservices_EU=
# #industrials_EU=
# #realsestate_EU=
# #materials_EU=
# #financials_EU=
# #consumercyclical_EU=


# Choose the Parameters

startdate=widgets.Text(
    value='2024-04-01',
    placeholder='yyyy-mm-dd',
    description='Start of Analysis',
    disabled=False   
)


enddate=widgets.Text(
    value=str(datetime.today().strftime("%Y-%m-%d")),
    placeholder='yyyy-mm-dd',
    description='End of Analysis',
    disabled=False   
)



display(startdate)
display(enddate)

Text(value='2024-04-01', description='Start of Analysis', placeholder='yyyy-mm-dd')

Text(value='2024-12-08', description='End of Analysis', placeholder='yyyy-mm-dd')

        ## Setup the generaL pORTFOLIO

In [6]:
# Setzet das Portfolio auf

def open_pdf(file_path):
    if os.path.exists(file_path):
        try:
            webbrowser.open(file_path)
        except Exception as e:
            print(f"An error occurred: {e}")
    else:
        print("File does not exist")
                    


def build_historic_metric(multdf,industry_name,MC,EQ):

        history_CW = pd.DataFrame(index=MC.index)
        history_EQ = pd.DataFrame(index=MC.index)
        for daysback in [5,10,21,50]:
            history_CW[str(daysback)]= MC["Average"]-MC["Average"].shift(daysback)
            history_EQ[str(daysback)]= EQ["Average"]-EQ["Average"].shift(daysback)
            
        history_EQ["EQ"]=0.4*history_EQ["50"]+0.3*history_EQ["21"]+0.15*history_EQ["10"]+0.05*history_EQ["5"]
        history_CW["CW"]=0.4*history_CW["50"]+0.3*history_CW["21"]+0.15*history_CW["10"]+0.05*history_CW["5"]

        hist = pd.DataFrame({'EQ': history_EQ["EQ"], "CW": history_CW["CW"]})
        multdf[industry_name]=hist



universe2="US_long"
#universe2="Deutschland500"

startdatum =startdate.value
enddatum=enddate.value
#Produce

universe = "styles"



the_portfolio=sg.portfolio(universe2)

Sectors = pd.read_csv(parentdir+"\\ETFS\\Mutterlisten\\DataBase\\US_long_database.csv", sep=";", encoding='latin-1').Sector.to_list()
#Industries = pd.read_csv(parentdir+"\\ETFS\\Mutterlisten\\DataBase\\Deutschland500_database.csv", sep=";", encoding='latin-1').industry.to_list()
Industries = pd.read_csv(parentdir+"\\ETFS\\Mutterlisten\\DataBase\\US_long_database.csv", sep=";", encoding='latin-1').Industry.to_list()
Sectors=np.unique(Sectors)
Industries=np.unique(Industries)
# produce

spy = sg.portfolio(universe,["SPY","RSP"])
spy.assets[0].read_grabbed_data()
spy.assets[1].read_grabbed_data()

spy_close = spy.matrix_of_grabbed_data("Close").truncate(before=startdatum, after = enddatum)

spy_factor=(spy_close.pct_change()+1).cumprod()-1

startstep=20
#spy_factor.plot()

In [15]:
matrix_close

,BXC,REZI
Date,,
2021-12-08,81.239998,26.680000
2021-12-09,83.050003,25.980000
2021-12-10,83.519997,25.930000
2021-12-13,80.620003,24.790001
2021-12-14,79.029999,24.719999
...,...,...
2024-12-02,126.720001,27.500000
2024-12-03,126.849998,27.750000
2024-12-04,128.630005,27.559999


## Plot Alle Industrygruppen vs. SPY (MARKETCAP und Equal wheighted)

In [8]:
# Plot ALLE Industrien vs. SPY
universe="US_long"
#universe="US_longVBT"
spy_factor_copy=spy_factor.truncate(before=startdatum)

IndustrieOfInterest = Industries

counter=0

####  build the multi columned dataframe
inds=np.repeat(Industries,2)
metrics=np.tile(["EQ","CW"],len(Industries)-1)
arr=[inds,metrics]
tuples=list(zip(*arr))
multiColumn = pd.MultiIndex.from_tuples(tuples, names=["Industry", "metric"])

multdf = pd.DataFrame(index=spy_factor_copy.index,columns=multiColumn)
#################################################################


days = [5,10,20,50,100]
RS = pd.DataFrame(index=["mcap"+str(days_back) for days_back in days])
RSe = pd.DataFrame(index=["eql"+str(days_back) for days_back in days])

### loop over industries
for industry in Industries:
    # Derive all tickers included in that Industry:
    
    liste_indus_stocks= the_portfolio.statics.loc[the_portfolio.statics["industry"]==industry].ticker.to_list()
    
    if len(liste_indus_stocks)>0:
            #Build the portfolio of tickers:
            industry_portfolio = sg.portfolio(universe,liste_indus_stocks)
            for asset  in industry_portfolio.assets:
                asset.read_grabbed_data()

            ### PLotte Industry vs. SPY

            sup_portfs=[industry_portfolio,]

            industry_marketCap=industry_portfolio.statics["marketCap"].sum()
            ### loop over single industry:
            for singel_pf in sup_portfs:
               ######################
                #### non- annualised retirns

                matrix_close= singel_pf.matrix_of_grabbed_data("Close").truncate(before=startdatum, after = enddatum)
                ### cum ulative returns
                matrix_close=matrix_close.bfill()
                matrix_factor=(matrix_close.pct_change()+1).cumprod()-1
                ### Sortiere 
               

                # bilde MC wheighted mean cum return of the industry 
                matrix_factor_marketcap_wheighted=matrix_factor.mul(industry_portfolio.statics["marketCap"],axis=1)
                matrix_factor_marketcap_wheighted=matrix_factor_marketcap_wheighted/industry_marketCap
                matrix_factor_marketcap_wheighted["Average"]=matrix_factor_marketcap_wheighted.sum(axis=1)

                
                # bilde EQUAL wheighted mean cum return of the industry 
                matrix_factor_EQUAL_wheighted=pd.DataFrame()
                matrix_factor_EQUAL_wheighted["Average"]=matrix_factor.sum(axis=1)/len(matrix_factor.columns)

                matrix_factor=matrix_factor.sort_values(matrix_factor.last_valid_index(), axis=1,ascending = False)
                
                build_historic_metric(multdf,industry,matrix_factor_marketcap_wheighted,matrix_factor_EQUAL_wheighted)


                names = matrix_factor.columns.to_list()
          
            
             ####  Berechne RS der Industry ##############
            
            values_mc=[matrix_factor_marketcap_wheighted.iloc[-1]["Average"]-\
                        matrix_factor_marketcap_wheighted.iloc[-days_back]["Average"] for days_back in days]
            values_eq=[matrix_factor_EQUAL_wheighted.iloc[-1]["Average"]-\
                        matrix_factor_EQUAL_wheighted.iloc[-days_back]["Average"] for days_back in days]
    
            RS[industry]=values_mc
            RSe[industry]=values_eq
        
                    #### non- annualised retirns


# dump the metric data into a json file 
pickle_output = parentdir+"\\OUTPUTS\\"+'metric.pkl'
with open(pickle_output, "wb") as file:
    pickle.dump(multdf,file)

RS=RS.transpose()
RSe=RSe.transpose()



C:\Users\r889990\AppData\Local\Temp\1\ipykernel_35468\4045266416.py:50: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  matrix_factor=(matrix_close.pct_change()+1).cumprod()-1
C:\Users\r889990\AppData\Local\Temp\1\ipykernel_35468\4045266416.py:50: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  matrix_factor=(matrix_close.pct_change()+1).cumprod()-1
C:\Users\r889990\AppData\Local\Temp\1\ipykernel_35468\4045266416.py:79: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns 

In [7]:
matrix_factor

NameError: name 'matrix_factor' is not defined

In [19]:
multdf.to_clipboard(decimal=",")



In [4]:

    
with open(pickle_output, 'rb') as file:
    loaded_dict = pickle.load(file)

In [5]:
loaded_dict.loc["2024-10-21",("Aerospace","CW")]

0.0657927438103143